In [13]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install -q sentence-transformers
!pip -q install tqdm

In [9]:
import json

with open("documents.json", "r") as f:
    documents_raw = json.load(f)

documents = []

for course_data in documents_raw:
    for document in course_data['documents']:
        document["course"] = course_data["course"]
        documents.append(document)

documents[:2]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'}]

In [14]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-mpnet-base-v2")

In [18]:
from tqdm import tqdm

for doc in tqdm(documents):
    doc['text_vector'] = model.encode(doc['text']).tolist()

100%|████████████████████████████████████████████████████████████████| 948/948 [06:21<00:00,  2.49it/s]


In [19]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")
es_client.info()

ObjectApiResponse({'name': '69c620c9be3e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'cwvJr1KeRAi49vbA8T7xYQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [22]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine",
            }
        }
    }
}
index_settings

{'settings': {'number_of_shards': 1, 'number_of_replicas': 0},
 'mappings': {'properties': {'text': {'type': 'text'},
   'section': {'type': 'text'},
   'question': {'type': 'text'},
   'course': {'type': 'keyword'},
   'text_vector': {'type': 'dense_vector',
    'dims': 768,
    'index': True,
    'similarity': 'cosine'}}}}

In [24]:
index_name = "course_questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [25]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████| 948/948 [00:17<00:00, 52.86it/s]


In [27]:
search_term = "windows, mac or linux?"
vector_search_term = model.encode(search_term)

query = {
    "field": "text_vector",
    'query_vector': vector_search_term,
    'k': 5,
    'num_candidates': 10000,
}


In [28]:
results = es_client.search(index=index_name, knn=query, source=['text', 'section', 'question', 'course'])

In [30]:
results['hits']['hits']

[{'_index': 'course_questions',
  '_id': 'F-lhmJEBgRn6KY0YWT-f',
  '_score': 0.78626704,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course_questions',
  '_id': 'sulhmJEBgRn6KY0YeECL',
  '_score': 0.6323599,
  '_source': {'question': "Any particular hardware requirements for the course or everything is mostly cloud? TIA! Couldn't really find this in the FAQ.",
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'For the Machine Learning part, all you need is a working laptop with an internet connection. The Deep Learning part is more resource intensive, but for that you can use a cloud (we use Saturn cloud but can be anything else).\n(Rileen Sinha; based on response 

In [36]:
results_adv = es_client.search(index=index_name, knn=query, size=5, explain=True, source=['text', 'section', 'question', 'course'], query={
    "match": {
        "course": "data-engineering-zoomcamp"
    }
})

In [37]:
results_adv['hits']['hits']

[{'_shard': '[course_questions][0]',
  '_node': 'Y1F1WT5oQ2akKOZJJzOTEw',
  '_index': 'course_questions',
  '_id': 'F-lhmJEBgRn6KY0YWT-f',
  '_score': 1.565181,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'},
  '_explanation': {'value': 1.565181,
   'description': 'sum of:',
   'details': [{'value': 0.78626704,
     'description': 'within top k documents',
     'details': []},
    {'value': 0.778914,
     'description': 'weight(course:data-engineering-zoomcamp in 35) [PerFieldSimilarity], result of:',
     'details': [{'value': 0.778914,
       'description': 'score(freq=1.0), computed as boost * idf * tf from:',
       'details': [{'value': 2.2, 'description': 'boost', 'details': []},
        {'value': 0.778914,
         'description